In [1]:
import sys
import os
spr_path = "/Users/Dixit/Documents/Studies/CU_Boulder/sem3/Independent_study/code/SpringRank/python"
sys.path.append(os.path.abspath(spr_path))
import SpringRank_tools as SR
import csv
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics


In [2]:
import networkx as nx
import numpy as np
import SpringRank_tools as sr
import tools as tl

In [3]:
input_data_dir = '/Users/Dixit/Documents/Studies/CU_Boulder/sem3/Independent_study/github/SpringRank/data/input/'
output_data_dir = '/Users/Dixit/Documents/Studies/CU_Boulder/sem3/Independent_study/github/SpringRank/data/output/'

In [4]:
def getGraph(indj):
    G = tl.build_graph_from_adjacency(indj)
    return G;

def graphProp(G):
    n=len(G.nodes)
    e=len(G.edges)
    print('no. of nodes= {0}, no. of edges= {1}'.format(n,e))

In [14]:
def formatChessFile(source,destination):
    with open(source,'r') as chess, open(destination, 'w') as out:
        chess = csv.reader(chess, delimiter=' ')
        writer = csv.writer(out, delimiter=' ')
        for row in chess:
            if '%' not in row:
                line=[]
                result = row[2].split('\t')
                if(result[0] == '1'):
                    line = [row[0],row[1],1]
                elif(result[0] == '-1'):
                    line = [row[1],row[0],1]
                if line:
                    writer.writerow(line)

def getChessGraphData(to_format):
    source = input_data_dir+'new_chess.data'
    destination = input_data_dir+'new_chess.data'
    if to_format: formatChessFile(source,destination)
    return getGraph(destination)

In [105]:
def run(G,alpha,l0,l1):
    nodes=list(G.nodes()) #  determines the order of the entries of matrix A
    A=nx.to_numpy_matrix(G,nodelist=nodes,weight='weight')
    '''
    Extracts SpringRank
    '''
    rank=sr.SpringRank(A,alpha=alpha,l0=l0,l1=l1)
    rank=tl.shift_rank(rank)   # (optional) shifts so that the min is in zero and the others are positive
    
    unordered_tuples=[(nodes[i],rank[i]) for i in range(G.number_of_nodes())]
    #ordered_x= sorted(rank, key=lambda tup: int(tup[0]),reverse=False)
    return rank,unordered_tuples

def save(sorted_tuples,alpha,l0,l1,G,file):
    '''
    Prints results
    '''
    print('SpringRank scores:')
    outfile=output_data_dir+'/'+file+'_SpringRank_'+'a'+str(alpha)+'_l0_'+str(l0)+'_l1_'+str(l1)+'.dat'
    outf=open(outfile,'w')

    for i in range(G.number_of_nodes()):
        outf.write("{} {}\n".format(sorted_tuples[i][0],sorted_tuples[i][1]))
        # print nodes[i],rank[i]
        #print(X[i][0],X[i][1])
    print('Results saved in:', outfile)
    outf.close()
    


In [690]:
def getEqn39(rank,A,start,end,step):
    x = np.arange(start,end,step)
    y=[]
    for xi in x:
        y.append(tl.eqs39(xi,rank,A))
    return x,y

def eqn39SimplePlot(rank,A,save):
    x,y = getEqn39(rank,A,0.1,20,1)
    plt.plot(x,y)
    plt.title('Eqn 39 :'+save)
    if save:
        plt.savefig(output_data_dir+save+'.svg')


def prediction(beta,G,scores):
    preds={}
    Adj = nx.to_numpy_matrix(G,nodelist=list(G.nodes))
    size,_=Adj.shape
    for i in range(0,size):
        for j in range(0,size):
            if(G.has_edge(i,j)):
                preds[(i,j)] = 1/(1+np.exp(-beta*2*(scores[i]-scores[j])))
            else: preds[(i,j)] = 0
    return preds


def getAccuracy(preds,G,Adj,num_edges):
    total=0
    size,_ = Adj.shape
    for i in range(0,size):
        for j in range(0,size):
            total += Adj[(i,j)]-((Adj[(i,j)]+Adj[(j,i)])*preds[(i,j)])
    total = 1-(0.5*(total)/num_edges)
    return total

 
#not using this
def getAccuracyOld(preds,G,Adj,num_edges):
    loss=0
    total=0
    A = nx.to_numpy_matrix(G,nodelist=list(G.nodes))
    for (i,j) in G.edges():
        Aij = G.number_of_edges(i,j)#['weight']
        #Aji = G.edges[(j,i)]['weight'] if G.has_edge(j,i) else 0
        Aji = G.number_of_edges(j,i)#['weight']
        loss+= Aij - ((Aij+Aji)*preds[(i,j)])
        #loss+= A[(i,j)] - ((A[(i,j)]+A[(j,i)])*preds[(i,j)])
    print(loss)
    total = 1-(0.5*(loss)/num_edges)
    return total
#not using this
def predictionOld(beta,G,_scores):
    preds = {}
    for i in range(len(_scores)):
        for j in range(len(_scores)):
            if(i!=j):
                if not (G.has_edge(i,j) or G.has_edge(j,i)):
                    preds[(i,j)] = 0
                else:
                    preds[(i,j)] = 1/(1+np.exp(-2*beta*(_scores[i]-_scores[j])))
    return preds


In [691]:
def expectedEqn(beta,c,si,sj):
    return c*np.exp(-beta*0.5*(si-sj-1)*(si-sj-1))

def getC(beta,num_nodes,scores):
    c = 20*num_nodes
    total = 0;
    for i in range(0,num_nodes):
        for j in range(0,num_nodes):
            total+=np.exp(-beta*0.5*(scores[i]-scores[j]-1)*(scores[i]-scores[j]-1))
    return c*1.0/(total*1.0)

def createNetwork(scores,beta,c,num_nodes):
    G = nx.MultiDiGraph()
    for i in range(0,num_nodes):
        for j in range(0,num_nodes):
            if(i!=j):
                G.add_node(i)
                G.add_node(j)
                mean = expectedEqn(beta,c,scores[i],scores[j])
                weight = np.random.poisson(mean)
                if weight>0:
                    G.add_edge(i,j,weight=weight)
    
    return G

def generateNetwork(beta):
    number_of_nodes=100
    mu, sigma = 0.5,1 # mean and standard deviation
    scores = np.random.normal(mu, sigma, number_of_nodes+1)
    c=getC(beta,number_of_nodes,scores)
    G = createNetwork(scores,beta,c,number_of_nodes)
    A = nx.to_numpy_matrix(G,nodelist=list(G.nodes),weight='weight')
    return G,A,scores

    
def getTrainingandTestSet(G):
    removed_edges = []
    edges_to_remove = list(G.edges)
    choice_idx = np.random.choice(len(edges_to_remove), int(len(edges_to_remove)*0.2), replace=False)
    for i in range(len(choice_idx)):
        a,b,_=edges_to_remove[choice_idx[i]]
        w = G.number_of_edges((a,b))
        #remove interaction, regardless of weight
        G.remove_edge(a,b)
        removed_edges.append((a,b,w))
        G.add_node(a)
        G.add_node(b)
    return G,removed_edges


   

# Create two networks 1 & 2 ; get training and test sets for network 1

In [692]:
G_1,A_1,scores1 = generateNetwork(beta=1) # Network A
G_2,A_2,scores2 = generateNetwork(beta=1) # Network B


In [693]:
graphProp(G_1)
graphProp(G_2)

no. of nodes= 100, no. of edges= 1798
no. of nodes= 100, no. of edges= 1690


In [694]:
#just saving original
G_1_copy = G_1.copy()
G_1_train,edges1_test = getTrainingandTestSet(G_1_copy) 

In [695]:
len(edges1_test)

359

In [696]:
graphProp(G_1)
graphProp(G_1_train)


no. of nodes= 100, no. of edges= 1798
no. of nodes= 100, no. of edges= 1439


# learn the ranks and get optimal beta for training set


In [697]:
alpha,l0,l1=0,0,1
rank1_train,tuples1_train = run(G_1_train,alpha,l0,l1)

A_1_train = nx.to_numpy_matrix(G_1_train,nodelist=list(G_1_train.nodes),weight='weight')

#get opt beta
temp1_train=tl.get_optimal_temperature(rank1_train,A_1_train)
beta1_train = 1/temp1_train
print((beta1_train))


0.6902809040117139


# get prediction and accuracy for the training set

In [698]:
preds1_train = prediction(beta1_train,G_1_train,rank1_train)
sigma1_train = getAccuracy(preds1_train,G_1_train,A_1_train,G_1_train.number_of_edges())

In [699]:
sigma1_train

0.81170933457928796

In [700]:
# Create a test graph from removed edges
G_1_test = nx.MultiDiGraph()
for(a,b,w) in edges1_test:
    G_1_test.add_edge(a,b,weight=1)
    
#some nodes might be lost while removing edges
for a in G_1.nodes():
    G_1_test.add_node(a)

In [701]:
graphProp(G_1_test)

no. of nodes= 100, no. of edges= 359


# Predict test set using trained parameters


In [702]:
alpha=0.
l0=0.
l1=1.    
A_1_test = nx.to_numpy_matrix(G_1_test,nodelist=list(G_1_test.nodes),weight='weight')
preds1_test = prediction(beta1_train,G_1_test,rank1_train)
sigma1_test = getAccuracy(preds1_test,G_1_test,A_1_test,G_1_test.number_of_edges())

In [703]:
sigma1_test

0.80050301171687277

# use learned ranks from network 1 to predict network 2 


In [704]:
preds2 = prediction(beta1_train,G_2,rank1_train)
sigma2 = getAccuracy(preds2,G_2,A_2,G_2.number_of_edges())

# this seems to be very high, why?


In [705]:
sigma2

0.744716941491119

In [706]:
sigma1_train

0.81170933457928796

# Train 100%

In [707]:
alpha=0.
l0=0.
l1=1.    
rank1,tuples1 = run(G_1,alpha,l0,l1)
A_1 = nx.to_numpy_matrix(G_1,nodelist=list(G_1.nodes),weight='weight')
temp1=tl.get_optimal_temperature(rank1,A_1)
beta1 = 1/temp1
print((beta1_train))


0.6902809040117139


In [708]:
preds1 = prediction(beta1,G_1,rank1)
sigma1 = getAccuracy(preds1,G_1,A_1,G_1.number_of_edges())

In [709]:
sigma1

0.8182914815541209

# Trying a small graph. Things we discussed during the meeting

In [710]:
#create  a toy graph
G_small = nx.MultiDiGraph()
G_small.add_edge(0,1)
G_small.add_edge(0,1)
G_small.add_edge(1,0)

0

In [711]:
G_small.edges

OutMultiEdgeView([(0, 1, 0), (0, 1, 1), (1, 0, 0)])

In [712]:
# learn ranks and get optimal temperature
alpha=0.
l0=0.
l1=1.    
rank1_small,tuples1_small = run(G_small,alpha,l0,l1)

A_1_small = nx.to_numpy_matrix(G_small,nodelist=list(G_small.nodes))
temp1_small=tl.get_optimal_temperature(rank1_small,A_1_small)
beta1_small = 1/temp1_small
print((beta1_small))


1.9235933878519509


In [713]:
A_1_small

matrix([[ 0.,  2.],
        [ 1.,  0.]])

In [714]:
preds_small = prediction(beta1_small,G_small,rank1_small)
sigma_small = getAccuracy(preds_small,G_small,A_1_small,G_small.number_of_edges())

In [715]:
sigma_small

1.0

In [716]:
preds_small

{(0, 0): 0,
 (0, 1): 0.92856093675019202,
 (1, 0): 0.071439063249808032,
 (1, 1): 0}

In [685]:
beta_a = np.log(2)*3/4

In [686]:
1/(1+np.exp(-4/3*beta_a))

0.66666666666666663

In [687]:
rank1_small

array([ 0.66666667,  0.        ])

In [688]:
G_small.edges

OutMultiEdgeView([(0, 1, 0), (0, 1, 1), (1, 0, 0)])

In [689]:
beta_a

0.51986038541995894